<a href="https://colab.research.google.com/github/ashishpatidar123/SURGE2023/blob/main/Model3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Input, BatchNormalization, ReLU, LeakyReLU, add, Conv3D,Reshape,Dense,Flatten
from keras.models import Model
from keras.layers import GlobalAvgPool3D
from keras.layers import multiply
import math
import time
import scipy.io as sio

from keras.callbacks import TensorBoard, Callback
import h5py

In [2]:
!pip install mat73
import mat73

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
reduction_ratio = 0.5

In [4]:
from keras.layers import GlobalAveragePooling3D

In [5]:
def attentionmodule(x, reduction_ratio=16):
    _, c, _, _, _ = x.shape

    # Squeeze
    y = GlobalAvgPool3D(data_format='channels_first')(x)
    y = Reshape((c,))(y)

    # Excitation
    y = Dense(c // reduction_ratio, activation='relu', use_bias=False)(y)
    y = Dense(c, activation='sigmoid', use_bias=False)(y)
    y = Reshape((c, 1, 1, 1))(y)
    y = tf.broadcast_to(y, tf.shape(x))

    return multiply([x, y])

In [6]:
y = Input(shape = (2,32,4,8))
channels = y.shape[1]
print(channels)
y = attentionmodule(y,16)
y.shape

2


TensorShape([None, 2, 32, 4, 8])

In [7]:
def network(x):

    def Block1(y):
        y = Conv3D(12,kernel_size = (3,2,2), padding = 'same',data_format='channels_first')(y)
        y = BatchNormalization()(y)
        y = LeakyReLU(alpha=0.1)(y)
        return y

    def Block2(y):
        y = Conv3D(24,kernel_size = (3,2,2),padding = 'same',data_format='channels_first')(y)
        temp = y
        y = BatchNormalization()(y)
        y = LeakyReLU(alpha=0.1)(y)
        temp = attentionmodule(temp,24)
        temp.shape
        y = multiply([temp,y])
        temp.shape
        return y

    def Block3(y):
        y = Conv3D(4, kernel_size = (3,2,2), padding = 'same',data_format='channels_first')(y)
        temp = y
        y = BatchNormalization()(y)
        y = LeakyReLU(alpha=0.1)(y)
        temp = attentionmodule(temp,4)
        y = multiply([temp,y])
        return y

    def Block4(y):
        y = Conv3D(1,kernel_size = (2,3,2), padding = 'same',data_format='channels_first')(y)
        y = BatchNormalization()(y)
        y = LeakyReLU(alpha=0.1)(y)
        y = Reshape((1024,))(y)
        y = Dense(2048, activation='linear')(y)
        y = BatchNormalization()(y)
        y = ReLU()(y)
        y = Reshape((2,32,8,4))(y)
        y = Conv3D(6,kernel_size = (3,2,2),padding = 'same',data_format='channels_first')(y)

        return y

    def residual(y):
        y = BatchNormalization()(y)
        y = LeakyReLU()(y)
        temp = y
        y = Conv3D(24,kernel_size = (3,2,2),padding = 'same',data_format='channels_first')(y)
        y = BatchNormalization()(y)
        y = LeakyReLU()(y)
        y = Conv3D(48,kernel_size = (3,2,2),padding = 'same',data_format='channels_first')(y)
        temp1 = y
        y = BatchNormalization()(y)
        y = LeakyReLU()(y)
        temp1 = attentionmodule(temp1,48)
        y = multiply([y,temp1])
        y = Conv3D(6,kernel_size = (3,2,2),padding = 'same',data_format='channels_first')(y)
        y = add([y,temp])
        return y

    def residual2(y):
        y = BatchNormalization()(y)
        y = LeakyReLU()(y)
        temp = y
        y = Conv3D(24,kernel_size = (5,3,3),padding = 'same',data_format='channels_first')(y)
        y = BatchNormalization()(y)
        y = LeakyReLU()(y)
        y = Conv3D(48,kernel_size = (5,3,3),padding = 'same',data_format='channels_first')(y)
        temp1 = y
        y = BatchNormalization()(y)
        y = LeakyReLU()(y)
        temp1 = attentionmodule(temp1,48)
        y = multiply([y,temp1])
        y = Conv3D(6,kernel_size = (5,3,3),padding = 'same',data_format='channels_first')(y)
        y = add([y,temp])
        return y

    def Block5(y):
        temp = y
        y = attentionmodule(y,6)
        y = multiply([temp,y])
        y = Conv3D(2,kernel_size = (3,2,2),padding = 'same',data_format='channels_first')(y)
        return y


    x = Block1(x)
    x = Block2(x)
    x = Block3(x)
    x = Block4(x)
    x = residual(x)
    x = residual2(x)
    x = Block5(x)

    return x

In [10]:
train_up_mat_file = sio.loadmat("/content/drive/MyDrive/Datasets/DATA_Htrainin_up_FDD100.mat")
train_down_mat_file = sio.loadmat("/content/drive/MyDrive/Datasets/DATA_Htrainin_down_FDD100.mat")
val_up_mat_file = sio.loadmat("/content/drive/MyDrive/Datasets/DATA_Hvalin_up_FDD100.mat")
val_down_mat_file = sio.loadmat("/content/drive/MyDrive/Datasets/DATA_Hvalin_down_FDD100.mat")

In [11]:
train_up = train_up_mat_file['HT']
train_down = train_down_mat_file['HT']
val_up = val_up_mat_file['HT']
val_down = val_down_mat_file['HT']

In [12]:
train_up = train_up.astype(float)
val_up = val_up.astype(float)
train_down = train_down.astype(float)
val_down = val_down.astype(float)

In [13]:
train_up.shape
train_down.shape
val_up.shape
val_down.shape

(15000, 2048)

In [14]:
train_up = np.reshape(train_up,(len(train_up),2,32,8,4))
train_down = np.reshape(train_down,(len(train_down),2,32,8,4))
val_up = np.reshape(val_up,(len(val_up),2,32,8,4))
val_down = np.reshape(val_down,(len(val_down),2,32,8,4))

In [15]:
train_up.shape
train_down.shape
val_up.shape
val_down.shape

(15000, 2, 32, 8, 4)

In [16]:
input_tensor = Input(shape=(2,32,8,4))
output_value = network(input_tensor)
# output_value = Conv3D(2, kernel_size=(1, 1, 1), padding='same', data_format='channels_first', activation='linear')(output_value)
# output_value = Conv3D(4, kernel_size=(1, 1, 1), padding='same', data_format='channels_last', activation='linear')(output_value)
output_value.shape

TensorShape([None, 2, 32, 8, 4])

In [17]:
initial_lr = 0.001
lr_drop_factor = 0.1


In [18]:
AcsiNet = Model(inputs=[input_tensor], outputs=[output_value])
AcsiNet.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=initial_lr), loss='mse')
print(AcsiNet.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 2, 32, 8, 4  0           []                               
                                )]                                                                
                                                                                                  
 conv3d (Conv3D)                (None, 12, 32, 8, 4  300         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 12, 32, 8, 4  16         ['conv3d[0][0]']                 
 alization)                     )                                                             

In [19]:
class LossHistory(Callback):
    # At the start of our training we intialize 2 variables to keep track of our losses
    def on_train_begin(self, logs={}):
        self.losses_train = []
        self.losses_val = []
    # We append to the end of the losses_train our current batch loss

    def on_batch_end(self, batch, logs={}):
        self.losses_train.append(logs.get('loss'))
    # We append to the end of the losses_val our validation loss

    def on_epoch_end(self, epoch, logs={}):
        self.losses_val.append(logs.get('val_loss'))


    def scheduler(epoch, lr):
        # Implement additional conditions if needed
        if epoch > 20 and lr > 1e-8:
            lr *= 0.1  # Reduce learning rate by a factor of 10
        return lr




In [20]:
history = LossHistory()
# Create a file name
file = 'AcsiNet'+time.strftime('_%m_%d')
path = 'result/TensorBoard_%s' % file
earlyStopping = tf.keras.callbacks.EarlyStopping(
    patience=200, restore_best_weights=True)

In [21]:
AcsiNet.fit(train_up,train_down,
           epochs=50,
           batch_size=512,
           validation_data=(val_up,val_down),
           shuffle=True,
           callbacks=[history,
                      TensorBoard(log_dir=path), earlyStopping])

Epoch 1/50
  6/137 [>.............................] - ETA: 49s - loss: 809.7615

137/137 [==============================] - 89s 464ms/step - loss: 37.4228 - val_loss: 0.2207
Epoch 2/50
137/137 [==============================] - 58s 424ms/step - loss: 0.2263 - val_loss: 0.2397
Epoch 3/50
137/137 [==============================] - 56s 411ms/step - loss: 0.1432 - val_loss: 9.4294
Epoch 4/50
137/137 [==============================] - 58s 424ms/step - loss: 0.1026 - val_loss: 4.7029
Epoch 5/50
137/137 [==============================] - 58s 426ms/step - loss: 0.0761 - val_loss: 0.1477
Epoch 6/50
137/137 [==============================] - 56s 411ms/step - loss: 0.0571 - val_loss: 0.0531
Epoch 7/50
137/137 [==============================] - 58s 425ms/step - loss: 0.0430 - val_loss: 0.0367
Epoch 8/50
137/137 [==============================] - 57s 416ms/step - loss: 0.0324 - val_loss: 0.0282
Epoch 9/50
137/137 [==============================] - 57s 419ms/step - loss: 0.0244 - val_loss: 0.0215
Epoch 10/50
137/137 [==============================] - 59s 428ms/step - loss: 0.018

In [22]:
filename = 'result/trainloss_%s.csv'%file
loss_history = np.array(history.losses_train)
np.savetxt(filename, loss_history, delimiter=",")

In [23]:
filename = 'result/valloss_%s.csv'%file
loss_history = np.array(history.losses_val)
np.savetxt(filename, loss_history, delimiter=",")

In [24]:
test_up_mat_file = sio.loadmat("/content/drive/MyDrive/Datasets/DATA_Htestin_up_FDD100.mat")
test_down_mat_file = sio.loadmat("/content/drive/MyDrive/Datasets/DATA_Htestin_down_FDD100.mat")

In [25]:
test_up = test_up_mat_file['HT']
test_down = test_down_mat_file['HT']

In [26]:
test_up = test_up.astype(float)
test_down = test_down.astype(float)

In [27]:
test_up = np.reshape(test_up,(len(test_up),2,32,8,4))
test_down = np.reshape(test_down,(len(test_down),2,32,8,4))


In [28]:
tStart = time.time()
x_hat = AcsiNet.predict(test_up)
tEnd = time.time()
print ("It cost %f sec" % ((tEnd - tStart)/test_up.shape[0]))

469/469 [==============================] - 5s 9ms/step
It cost 0.000777 sec


In [29]:
def NMSE(x_i,x_hat):
    x_i_real = np.reshape(x_i[:,:,:,0],(len(x_i),-1))
    x_i_img  = np.reshape(x_i[:,:,:,1],(len(x_i),-1))
    x_i_C = x_i_real-0.5 + 1j*(x_i_img-0.5)

    x_hat_real = np.reshape(x_hat[:,:,:,0],(len(x_i),-1))
    x_hat_img  = np.reshape(x_hat[:,:,:,1],(len(x_i),-1))
    x_hat_C = x_hat_real-0.5 + 1j*(x_hat_img-0.5)

    mse = np.mean(np.abs(x_i_C-x_hat_C)**2, axis=1)
    power = np.sum(np.abs(x_i_C)**2, axis=1)
    #NMSE = mse/power
    #NMSE = 10*np.log10(NMSE)
    #NMSE = np.sum(NMSE)
    NMSE = 10 * np.log10(np.mean(mse) / np.maximum(np.mean(power), 1e-10))
    return NMSE

In [30]:
print("NMSE is ", NMSE(test_down,x_hat))
filename = "result/decoded_%s.csv" %file
x_hat1 = np.reshape(x_hat, (len(x_hat), -1))
np.savetxt(filename, x_hat1, delimiter=",")
print(AcsiNet.summary())

NMSE is  -24.53458348259932
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 2, 32, 8, 4  0           []                               
                                )]                                                                
                                                                                                  
 conv3d (Conv3D)                (None, 12, 32, 8, 4  300         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 12, 32, 8, 4  16         ['conv3d[0][0]']                 
 alization)                     )                                 

In [31]:
# Serialize model to JSON
model_json = AcsiNet.to_json()
outfile = "result/model3%s.json"%file
with open(outfile, "w") as json_file:
    json_file.write(model_json)

# Serialize weights to HDF5
outfile = "result/model3%s.h5"%file
AcsiNet.save_weights(outfile)

# Save model in tensorflow format
AcsiNet.save("tensorflow/model3%s"%file)